<a href="https://colab.research.google.com/github/samula98/proj5_v0/blob/main/proj5_v00_NLP_Description_for_Students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing



This project will give you practical experience using Natural Language Processing techniques. This project is in three parts:
- in part 1) you will use a traditional dataset in a CSV file
- in part 2) you will use the Wikipedia API to directly access content
on Wikipedia.
- in part 3) you will make your notebook interactive


## Part 1)



- The CSV file is available at https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv
- The file contains a list of famous people and a brief overview.
- The goal of part 1) is provide the capability to
  - Take one person from the list as input and output the 10 other people who's overview are "closest" to the person in a Natural Language Processing sense
  - Also output the sentiment of the overview of the person



In [1]:
import numpy as np
import pandas as pd
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import NearestNeighbors
import random
import re
import nltk
# nltk.download('omw-1.4')
nltk.download('punkt_tab')
# nltk.download('averaged_perceptron_tagger_eng')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
import textwrap

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
! python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [3]:
url = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv'
df = pd.read_csv(url)

### Visualizing the data

In [4]:
df.shape

(42786, 3)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42786 entries, 0 to 42785
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URI     42786 non-null  object
 1   name    42786 non-null  object
 2   text    42786 non-null  object
dtypes: object(3)
memory usage: 1002.9+ KB


In [6]:
df.head(10)

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...
5,<http://dbpedia.org/resource/Sam_Henderson>,Sam Henderson,sam henderson born october 18 1969 is an ameri...
6,<http://dbpedia.org/resource/Aaron_LaCrate>,Aaron LaCrate,aaron lacrate is an american music producer re...
7,<http://dbpedia.org/resource/Trevor_Ferguson>,Trevor Ferguson,trevor ferguson aka john farrow born 11 novemb...
8,<http://dbpedia.org/resource/Grant_Nelson>,Grant Nelson,grant nelson born 27 april 1971 in london also...
9,<http://dbpedia.org/resource/Cathy_Caruth>,Cathy Caruth,cathy caruth born 1955 is frank h t rhodes pro...


In [7]:
df.isnull().sum().sum()

0

In [8]:
df.describe().transpose()

,count,unique,top,freq
URI,42786,42786,<http://dbpedia.org/resource/Digby_Morrell>,1
name,42786,42785,author),2
text,42786,42786,digby morrell born 10 october 1979 is a former...,1


In [9]:
print(df.duplicated().value_counts()) # no duplicated data

False    42786
Name: count, dtype: int64


In [33]:
pd.set_option('display.max_colwidth', None)
print(df.iloc[0]['text'])

digby morrell born 10 october 1979 is a former australian rules footballer who played with the kangaroos and carlton in the australian football league aflfrom western australia morrell played his early senior football for west perth his 44game senior career for the falcons spanned 19982000 and he was the clubs leading goalkicker in 2000 at the age of 21 morrell was recruited to the australian football league by the kangaroos football club with its third round selection in the 2001 afl rookie draft as a forward he twice kicked five goals during his time with the kangaroos the first was in a losing cause against sydney in 2002 and the other the following season in a drawn game against brisbaneafter the 2003 season morrell was traded along with david teague to the carlton football club in exchange for corey mckernan he played 32 games for the blues before being delisted at the end of 2005 he continued to play victorian football league vfl football with the northern bullants carltons vflaf

In [11]:
# viewing the first person's text and wrapping it to make it easy to read
pd.set_option('display.max_colwidth', None)
person_1 = df.loc[df.index[0], 'text']
person_1w = textwrap.fill(person_1, width = 80)
print(person_1w)

digby morrell born 10 october 1979 is a former australian rules footballer who
played with the kangaroos and carlton in the australian football league aflfrom
western australia morrell played his early senior football for west perth his
44game senior career for the falcons spanned 19982000 and he was the clubs
leading goalkicker in 2000 at the age of 21 morrell was recruited to the
australian football league by the kangaroos football club with its third round
selection in the 2001 afl rookie draft as a forward he twice kicked five goals
during his time with the kangaroos the first was in a losing cause against
sydney in 2002 and the other the following season in a drawn game against
brisbaneafter the 2003 season morrell was traded along with david teague to the
carlton football club in exchange for corey mckernan he played 32 games for the
blues before being delisted at the end of 2005 he continued to play victorian
football league vfl football with the northern bullants carltons vflaf

In [23]:
clean_blob_1 = person_1.lower()
for c in ["\n", "\s", "'", "  "]:
  clean_blob_1 = person_1.replace(c, " ")
clean_blob_1

'digby morrell born 10 october 1979 is a former australian rules footballer who played with the kangaroos and carlton in the australian football league aflfrom western australia morrell played his early senior football for west perth his 44game senior career for the falcons spanned 19982000 and he was the clubs leading goalkicker in 2000 at the age of 21 morrell was recruited to the australian football league by the kangaroos football club with its third round selection in the 2001 afl rookie draft as a forward he twice kicked five goals during his time with the kangaroos the first was in a losing cause against sydney in 2002 and the other the following season in a drawn game against brisbaneafter the 2003 season morrell was traded along with david teague to the carlton football club in exchange for corey mckernan he played 32 games for the blues before being delisted at the end of 2005 he continued to play victorian football league vfl football with the northern bullants carltons vfla

In [35]:
person_1tb = TextBlob(person_1)
person_1s = [x.singularize() for x in person_1tb.words]
person_1c = ' '.join(person_1s)
person_1c

'digby morrell born 10 october 1979 is a former australian rule footballer who played with the kangaroo and carlton in the australian football league aflfrom western australium morrell played hi early senior football for west perth hi 44game senior career for the falcon spanned 19982000 and he wa the club leading goalkicker in 2000 at the age of 21 morrell wa recruited to the australian football league by the kangaroo football club with it third round selection in the 2001 afl rookie draft a a forward he twice kicked five goal during hi time with the kangaroo the first wa in a losing cause against sydney in 2002 and the other the following season in a drawn game against brisbaneafter the 2003 season morrell wa traded along with david teague to the carlton football club in exchange for corey mckernan he played 32 game for the blue before being delisted at the end of 2005 he continued to play victorian football league vfl football with the northern bullant carlton vflaffiliate in 2006 an

In [ ]:
persons =

In [29]:
pd.set_option('display.max_colwidth', None)
print(df.iloc[:42786]['text'])

0        (d, i, g, b, y,  , m, o, r, r, e, l, l,  , b, o, r, n,  , 1, 0,  , o, c, t, o, b, e, r,  , 1, 9, 7, 9,  , i, s,  , a,  , f, o, r, m, e, r,  , a, u, s, t, r, a, l, i, a, n,  , r, u, l, e, s,  , f, o, o, t, b, a, l, l, e, r,  , w, h, o,  , p, l, a, y, e, d,  , w, i, t, h,  , t, h, e,  , k, a, n, g, a, ...)
1        (a, l, f, r, e, d,  , j,  , l, e, w, y,  , a, k, a,  , s, a, n, d, y,  , l, e, w, y,  , g, r, a, d, u, a, t, e, d,  , f, r, o, m,  , u, n, i, v, e, r, s, i, t, y,  , o, f,  , c, h, i, c, a, g, o,  , i, n,  , 1, 9, 7, 3,  , a, f, t, e, r,  , s, t, u, d, y, i, n, g,  , p, s, y, c, h, i, a, t, r, y,  , ...)
2        (h, a, r, p, d, o, g,  , b, r, o, w, n,  , i, s,  , a,  , s, i, n, g, e, r,  , a, n, d,  , h, a, r, m, o, n, i, c, a,  , p, l, a, y, e, r,  , w, h, o,  , h, a, s,  , b, e, e, n,  , a, c, t, i, v, e,  , i, n,  , c, a, n, a, d, a, s,  , b, l, u, e, s,  , s, c, e, n, e,  , s, i, n, c, e,  , 1, 9, 8, 2, ...)
3        (f, r, a, n, z,  , r, o, t, t, e, n, s, t, e, 

In [20]:
# person_1wl = ' '.join([word for word in person_1w.split() if word.lower() not in stop_words])
# person_1wl

### Cleaning the data

In [12]:
# def clean_blob (text):
#   '''
#   This function makes the text all lower case. It also uses Regular Expressions to take out spaces, pluralization, and possession from the text.
#   '''
#   raboof = re.compile(r"(\n|'s|'| )+")
#   df = re.sub(raboof, ' ', df.lower())
#   return text_clean

In [13]:
# ## my version that does not appear to work:


# def clean_blob (text):
#   '''
#   This function makes the text all lower case. It also uses Regular Expressions to take out spaces, pluralization, and possession from the text.
#   '''
#   # text = text.lower()
#   raboof = re.compile(r"(\n|'s|'| ')+")
#   text_clean = re.sub(raboof, ' ', text_clean.lower())
#   text_clean = TextBlob(text)
#   return text_clean

In [14]:
## FROM WIL:


def process_data (text):
  '''
  This function will take a value and apply the lower case method, remove white spaces and characters and convert the text to TextBlob
  '''
  text = text.lower()
  text = re.sub(r"[^\w\s]", "", text)
  text = TextBlob(text)
  return text

In [16]:
process_data

<function __main__.process_data(text)>

In [18]:
df['text'] = df['text'].apply(process_data)

## Part 2)



- For the same person from step 1), use the Wikipedia API to access the whole content of that person's Wikipedia page.
- The goal of part 2) is to produce the capability to:
  1. For that Wikipedia page determine the sentiment of the entire page
  1. Print out the Wikipedia article
  1. Collect the Wikipedia pages from the 10 nearest neighbors in Step 1)
  1. Determine the nearness ranking of these 10 to your main subject based on their entire Wikipedia page
  1. Compare the nearest ranking from Step 1) with the Wikipedia page nearness ranking



In [ ]:
!pip3 install wikipedia-api

In [ ]:
# df_orig.ilog[3]['text']

## Part 3)


Make an interactive notebook.

In addition to presenting the project slides, at the end of the presentation each student will demonstrate their code using a famous person suggested by the other students that exists in the DBpedia set.


In [ ]:
!curl -s https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv | wc -l